notes : 
- trouver comment exploiter le "superuser"

## Traitement NLP

In [88]:
import pandas as pd
import json

In [89]:
data = []
with open("data/philly_restaurant_reviews.json", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
textes=pd.DataFrame(data)

In [90]:
textes.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
1,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
2,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
3,oyaMhzBSwfGgemSGuZCdwQ,Dd1jQj7S-BFGqRbApFzCFw,YtSqYv1Q_pOltsVPSx54SA,5.0,0,0,0,Tremendous service (Big shout out to Douglas) ...,2013-06-24 11:21:25
4,Xs8Z8lmKkosqW5mw_sVAoA,IQsF3Rc6IgCzjVV9DE8KXg,eFvzHawVJofxSnD7TgbZtg,5.0,0,0,0,My absolute favorite cafe in the city. Their b...,2014-11-12 15:30:27


Nous n’avons volontairement pas concaténé l’ensemble des avis d’un restaurant afin d’éviter un biais de volume lié au nombre d’avis. 

In [91]:
import pandas as pd

# nombre d'avis par business
nb_avis = textes.groupby("business_id").size()

nb_avis.describe()


count    5852.000000
mean      117.445147
std       247.806219
min         5.000000
25%        14.000000
50%        40.000000
75%       118.000000
max      5778.000000
dtype: float64

Nous prenons 40 avis par restaurant ce qui correspond à la mediane du nombre d'avis par restaurant. Nous prenons les 40 avis les plus recents pour avoir l'information la plus a jour.

In [92]:
# s'assurer que la colonne date est bien au format datetime
textes["date"] = pd.to_datetime(textes["date"])

textes_restaurant = (
    textes
    .sort_values("date", ascending=False)   # du plus récent au plus ancien
    .groupby("business_id")
    .head(40)                                # garder les 40 plus récents
    .groupby("business_id", as_index=False)["text"]
    .apply(" ".join)
)

textes_restaurant.head()

,business_id,text
0,-0M0b-XhtFagyLmsBtOe8w,Ok so this place is an extremely odd hole in t...
1,-0PN_KFPtbnLQZEeb23XiA,Far and away the best Chinese food to order if...
2,-0TffRSXXIlBYVbb5AwfTg,I went to indieblue a few years ago for happy ...
3,-0eUa8TsXFFy0FCxHYmrjg,"We are sitting in an ICU waiting room, several..."
4,-1B9pP_CrRBJYPICE5WbRA,The food is absolutely amazing. I get food ...


Conformément à la méthodologie proposée par BERTopic, les documents sont représentés à l’aide d’embeddings Sentence-BERT (SBERT), une approche spécifiquement conçue pour comparer des phrases et des documents entiers sur la base de leur similarité sémantique. d'apres les auteurs it achieves state-of-the-art performance on various sentence embedding tasks (Reimers and urevych, 2020; Thakur et al., 2020). En particulier, le modèle all-MiniLM-L6-v2 est recommandé dans des contextes à ressources limitées sans GPU

notes : donner + d'argument prk sbert bon pour notre cas

Sbert doit recevoir un texte le plus priche du langage naturel donc les preprocessing lourds comme les stopword removal ou la lemmatization sont appliqués suelement dans la partie topic representations (c-TF-IDF) suivant les recos (Grootendorst (2022) and Reimers & Gurevych (2019).)

par contre on sait que naturlmment nous allons avoir des clusters autour du type de nourriturz comment on l'a vu dans le papier... et on a fait le test et on avait bcp de mots liés à la cuisine. donc on- va les enelever. 

In [93]:
import re
import html

FOOD_STOPWORDS = {
    # plats / items
    "pizza", "pasta", "burger", "burrito", "taco", "tacos",
    "sushi", "ramen", "noodles", "steak", "salad", "sandwich", "soup",
    "dumpling", "dumplings",
    "naan", "masala", "tikka", "curry", "pad",
    "falafel", "hummus", "pita",
    "cheesesteak", "hoagie",
    "cake", "donut", "cupcake",
    "coffee", "latte", "espresso",
    "seafood", "shrimp", "lobster", "crab",
    "fries",

    # ingrédients
    "chicken", "beef", "pork", "fish", "rice", "cheese",

    # cuisines / origines
    "italian", "french", "thai", "chinese", "japanese",
    "mexican", "indian", "korean", "vietnamese",
    "greek", "spanish", "turkish"
}

def clean_for_embedding(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Decode HTML entities
    text = html.unescape(text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", " ", text)

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)

    # Keep only letters and spaces
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    # Lowercase
    text = text.lower()

    # Remove food-related terms (key step)
    tokens = [
        tok for tok in text.split()
        if tok not in FOOD_STOPWORDS
    ]

    # Normalize whitespace
    text = " ".join(tokens)
    text = re.sub(r"\s+", " ", text).strip()

    return text
textes_restaurant["cleaned_text_embedding"] = textes_restaurant["text"].apply(clean_for_embedding)

On peut passer maintenant àl'embedding 

In [94]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = textes_restaurant["cleaned_text_embedding"].tolist()
embeddings_txt = model.encode(
    texts,
    batch_size=32,          
    show_progress_bar=True,
    normalize_embeddings=True  #ce sera pratique pour le clustering plus tard
)

Batches:   0%|          | 0/183 [00:00<?, ?it/s]

In [95]:
embeddings_txt

array([[ 0.07339897, -0.02592888,  0.04090234, ..., -0.01813973,
        -0.08050999, -0.04733944],
       [ 0.02709788, -0.00823581,  0.05498442, ..., -0.00430089,
        -0.08237864,  0.02010359],
       [ 0.04485257, -0.04540356,  0.06240261, ..., -0.01022023,
        -0.07408096, -0.02439085],
       ...,
       [-0.03398207, -0.0067131 ,  0.07501908, ..., -0.00645372,
        -0.07401868, -0.02476408],
       [-0.01100712, -0.03541555,  0.05553192, ...,  0.01721645,
         0.01522463, -0.0668183 ],
       [ 0.04314486,  0.022509  ,  0.12038537, ..., -0.02080076,
        -0.06830619,  0.01881207]], dtype=float32)

1) ajouter autres features (5/6 max)
2)Réduction de dimension (UMAP)
3)Clustering (HDBSCAN)

4)Extraction des mots-clés (c-TF-IDF)

## Reduction de dimensionalité

On utilise la methode UMAP pour la reduction de dimensionnalité “Although PCA and t-SNE are well-known methods for reducing dimensionality, UMAP has shown to preserve more of the local and global features of high-dimensional data in lower projected dimensions.” BERTopic

on ne peut pas optimiser les hyperparametres dans du non supervisé donc pour choisir le hyperparametres on se base sur bertopic, ils ont testé sur plusieurs textes et ont choisir ces params. on verifiera plus tard la robustesse.

notes : chercher ce que signifie chaque param

In [96]:
import umap

umap_model = umap.UMAP(
    n_neighbors=30,
    min_dist=0.00,
    n_components=5,
    metric="cosine", # a changer apres quand on ajoutes les features numeriques par euclidean
    random_state=42
)

X_umap = umap_model.fit_transform(embeddings_txt)


## Clustering

On utilise HDBscan pour effectuer la clusterisation, car deja ca focntionne bien apres umap et c'est plsu ronuste. aussi bertopic l'ont choisit lol (à justifier un peu plus)
justifier le choix d'hyperparam


In [97]:
import hdbscan

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=50, #taille minimale d'un cluster, on met 50 puis on verra 
    min_samples=50, #en pratique on met min_samples = min_cluster_size pour des resultats plus stables
    metric="euclidean", # parce que UMAP nous a donné des vecteurs en euclidean
    cluster_selection_method="eom", #on garde les clusters qui ont le plus de “masse” stable dans l’arbre, leaf npous donnerait des clusters trop petit
    prediction_data=True #pour pouvoir assigner de nouveaux points plus tard
)

cluster_labels = hdbscan_model.fit_predict(X_umap)


In [98]:
print(cluster_labels)

[10  1 10 ... -1 -1  6]


## interpretation des clusters

on commence par attacher les clusters aux textes

In [99]:
textes_restaurant['cluster'] = cluster_labels

In [100]:
textes_restaurant

,business_id,text,cleaned_text_embedding,cluster
0,-0M0b-XhtFagyLmsBtOe8w,Ok so this place is an extremely odd hole in t...,ok so this place is an extremely odd hole in t...,10
1,-0PN_KFPtbnLQZEeb23XiA,Far and away the best Chinese food to order if...,far and away the best food to order if you liv...,1
2,-0TffRSXXIlBYVbb5AwfTg,I went to indieblue a few years ago for happy ...,i went to indieblue a few years ago for happy ...,10
3,-0eUa8TsXFFy0FCxHYmrjg,"We are sitting in an ICU waiting room, several...",we are sitting in an icu waiting room several ...,-1
4,-1B9pP_CrRBJYPICE5WbRA,The food is absolutely amazing. I get food ...,the food is absolutely amazing i get food deli...,4
...,...,...,...,...
5847,zxY4DgtXsVHihSUpsmwamg,This Place has Good Hoagies and the Staff is s...,this place has good hoagies and the staff is s...,6
5848,zy7uNOvpykrq-XlmDY_wHA,"Sadiki's is now Relish, nothing last's over th...",sadiki s is now relish nothing last s over the...,-1
5849,zyMkbavgHASQtqVwaock9A,I've had pretty much everything on the menu an...,i ve had pretty much everything on the menu an...,-1
5850,zz-fcqurtm77bZ_rVvo2Lw,"My love affair with Yumtown began with ""The Jo...",my love affair with yumtown began with the joy...,-1


In [101]:
(textes_restaurant["cluster"] == -1).sum()


3154

ca fait 30% d'outliers, c'est beaucoup j'ai l'impression ? peut etre reduire le nbre de documents minimum par cluster

Le but de cette prochaine etape est de labeliser les clusters. pour cela on va dabord faire un nettoyage des textes plus important que pour le Sbert. En effet sbert comprends le sens total de la phrase, la pou c-TFIDF travailles avec des mots discrimiannt.
Etant donné que nous on ne veut pas avoir des clusters qui se concentrent sur le type de nourriture mais sur l'experience on va enlever les mots relatifs aux plats, pour eviter d'avoir des cluysters de type "chinease food" "indian food" ect 

In [103]:
import re
import html
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stoplist = set(stopwords.words("english"))
porter = PorterStemmer()
FOOD_STOPWORDS = {
    # plats / items
    "pizza", "pasta", "burger", "burrito", "taco", "tacos",
    "sushi", "ramen", "noodles", "steak", "salad", "sandwich", "soup",
    "dumpling", "dumplings",
    "naan", "masala", "tikka", "curry", "pad",
    "falafel", "hummus", "pita",
    "cheesesteak", "hoagie",
    "cake", "donut", "cupcake",
    "coffee", "latte", "espresso",
    "seafood", "shrimp", "lobster", "crab",
    "fries",

    # ingrédients
    "chicken", "beef", "pork", "fish", "rice", "cheese",

    # cuisines / origines
    "italian", "french", "thai", "chinese", "japanese",
    "mexican", "indian", "korean", "vietnamese",
    "greek", "spanish", "turkish"
}
stoplist = stoplist.union(FOOD_STOPWORDS)


def clean_for_ctfidf(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text)

    tokens = [
        porter.stem(tok)
        for tok in text.split()
        if tok not in stoplist and len(tok) > 2
    ]

    return " ".join(tokens)
textes_restaurant["cleaned_text_ctfidf"] = textes_restaurant["text"].apply(clean_for_ctfidf)

on fait un document avec tous les textes par cluster

In [104]:
df = textes_restaurant.copy()

docs_per_cluster = (
    df[df["cluster"] != -1]
    .groupby("cluster")["cleaned_text_ctfidf"]
    .apply(" ".join)
)


on calcul le c-tf-idf

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# vectorisation
vectorizer = CountVectorizer(
    ngram_range=(1, 3),
    min_df=5,
    stop_words="english"
)

X = vectorizer.fit_transform(docs_per_cluster.values)

# c-TF-IDF
tf = X.toarray()
tf_sum = tf.sum(axis=0)

A = tf.sum(axis=1).mean()

ctfidf = tf * np.log(1 + A / tf_sum)


on extrait les tops mots pas cluster

In [106]:
terms = vectorizer.get_feature_names_out()

top_words = {}

for i, cluster_id in enumerate(docs_per_cluster.index):
    scores = ctfidf[i]
    top_idx = scores.argsort()[::-1][:10]
    top_words[cluster_id] = [(terms[j], scores[j]) for j in top_idx]


In [107]:
for cluster_id, words in top_words.items():
    print(f"\n🟦 Cluster {cluster_id}")
    for word, score in words:
        print(f"  {word:<25} {score:.3f}")



🟦 Cluster 0
  vegan                     9121.134
  food                      4939.191
  good                      3854.357
  place                     3818.523
  order                     3292.073
  tri                       3282.370
  great                     3262.118
  love                      3089.146
  delici                    2997.990
  like                      2837.979

🟦 Cluster 1
  food                      28754.853
  order                     23990.038
  place                     21493.280
  good                      20749.405
  roll                      16997.314
  like                      15843.372
  time                      14950.833
  restaur                   14642.778
  tri                       14642.122
  great                     13963.361

🟦 Cluster 2
  food                      7536.471
  good                      5541.074
  place                     5342.325
  order                     5198.237
  salsa                     4370.295
  great                   